In [ ]:
-- which movie collected top 3 box office in this whole time

select
mn.name,
m.released,
concat('$',round((m.gross/1000000),0),' M') boxOffice
from moviedb.fact.fct_movies m
left join moviedb.dim.dim_movie_name mn
on m.m_id = mn.m_id
order by gross desc limit 3;

| NAME               | RELEASED                          | BOXOFFICE |
|--------------------|-----------------------------------|-----------|
| Avatar             | December 18, 2009 (United States) | $2847 M   |
| Avengers: Endgame  | April 26, 2019 (United States)    | $2798 M   |
| Titanic            | December 19, 1997 (United States) | $2202 M   |


In [ ]:
-- Which actor-director duo has the highest average box office gross?

select 
s.star,
d.director,
concat('$',round((avg(gross)/1000000),0),' M') avgGross
from moviedb.fact.fct_movies m
left join moviedb.dim.dim_director d on m.d_id = d.d_id
left join moviedb.dim.dim_star s on m.s_id=s.s_id
group by s.star, d.director
order by avg(gross) desc
limit 5;

| STAR               | DIRECTOR        | AVGGROSS |
|--------------------|-----------------|----------|
| Sam Worthington    | James Cameron   | $2847 M  |
| Robert Downey Jr.  | Anthony Russo   | $2423 M  |
| Leonardo DiCaprio  | James Cameron   | $2202 M  |
| Donald Glover      | Jon Favreau     | $1671 M  |
| Chris Pratt        | Colin Trevorrow | $1671 M  |


In [ ]:
-- Which writer has the highest number of films with a perfect score (8 and above)

select distinct w.writer, count(m.m_id) no_of_movies,round(avg(m.score),1) avg_score from moviedb.fact.fct_movies m
left join moviedb.dim.dim_writer w on m.w_id=w.w_id
where m.score >= 8
group by w.writer
order by count(m.m_id) desc
limit 5;

| WRITER            | NO_OF_MOVIES | AVG_SCORE |
|-------------------|--------------|-----------|
| Quentin Tarantino | 6            | 8.3       |
| Hayao Miyazaki    | 5            | 8.3       |
| Jonathan Nolan    | 4            | 8.6       |
| Stephen King      | 4            | 8.6       |
| Pete Docter       | 3            | 8.1       |


In [ ]:
-- Which country produces the highest number of horror films and what is their total budget spent on those films?

-- Horror id = 4

select distinct c.country,count(m.m_id) no_of_movies,
concat('$',round((sum(m.budget) / 1000000),0),' M') amount_spend
from moviedb.fact.fct_movies m
left join moviedb.dim.dim_country c on m.p_id=c.p_id
where m.g_id = 4
group by c.country
order by count(m.m_id) desc limit 3;

| COUNTRY        | NO_OF_MOVIES | AMOUNT_SPEND |
|----------------|--------------|--------------|
| United States  | 246          | $2701 M      |
| United Kingdom | 24           | $287 M       |
| Canada         | 23           | $267 M       |


In [ ]:
-- For each movie rating, which production company has the highest box office gross (using Windows func)

with cte_1 as (
select 
r.rating rating, 
c.company studios, 
sum(m.gross) gross
from moviedb.fact.fct_movies m
left join moviedb.dim.dim_rating r on m.r_id=r.r_id
left join moviedb.dim.dim_company c on m.c_id=c.c_id
where r.rating not in ('','Not Rated','Unrated')
group by rating, company
), cte_2 as (select *,
row_number() over(partition by rating order by gross desc) ranking
from cte_1
), cte_3 as (select rating,studios,concat('$',round((gross/1000000),0),' M') boxOffice
from cte_2 where ranking = 1 )
select rating, studios, boxOffice from cte_3;

| RATING  | STUDIOS                                                        | BOXOFFICE |
|---------|----------------------------------------------------------------|-----------|
| PG-13   | Warner Bros.                                                   | $30886 M  |
| NC-17   | Haishang Films                                                 | $67 M     |
| G       | Walt Disney Pictures                                           | $8657 M   |
| TV-PG   | Amuse                                                          | $358 M    |
| R       | Warner Bros.                                                   | $15717 M  |
| TV-MA   | Beijing Dengfeng International Culture Communications Company  | $700 M    |
| PG      | Walt Disney Pictures                                           | $17705 M  |
| Approved| Metro-Goldwyn-Mayer (MGM)                                      | $37 M     |
| TV-14   | Premiere Picture                                               | $6 M      |
| X       | Carolco International N.V.                                     | $17 M     |
